# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 04:16:10] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=39993, nccl_port=None, mem_fraction_static=0.849, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=499941484, constrained_json_w

[2025-07-09 04:16:10] Inferred chat template from model path: llama-2


[2025-07-09 04:16:18] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 04:16:18] Init torch distributed begin.


[2025-07-09 04:16:18] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 04:16:20] Load weight begin. avail mem=41.53 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.25s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.25s/it]

[2025-07-09 04:16:25] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=5.91 GB, mem usage=0.93 GB.
[2025-07-09 04:16:25] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-07-09 04:16:25] Memory pool end. avail mem=5.59 GB


[2025-07-09 04:16:26] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=5.82 GB


[2025-07-09 04:16:26] INFO:     Started server process [1405521]
[2025-07-09 04:16:26] INFO:     Waiting for application startup.
[2025-07-09 04:16:26] INFO:     Application startup complete.
[2025-07-09 04:16:26] INFO:     Uvicorn running on http://127.0.0.1:39993 (Press CTRL+C to quit)


[2025-07-09 04:16:26] INFO:     127.0.0.1:56672 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 04:16:27] INFO:     127.0.0.1:56680 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 04:16:27] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T04:16:27.621492


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-09 04:17:32] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, #token: 7, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-09T04:17:32.130710


[2025-07-09 04:18:02] INFO:     127.0.0.1:56696 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 04:18:02] The server is fired up and ready to roll!


[2025-07-09 04:18:04] INFO:     127.0.0.1:56704 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

[2025-07-09 04:18:04] Child process unexpectedly failed with exitcode=9. pid=1406156
[2025-07-09 04:18:04] Child process unexpectedly failed with exitcode=9. pid=1406089


### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 04:18:11] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=31605, nccl_port=None, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=77120696, constrained_json_whit

[2025-07-09 04:18:12] Inferred chat template from model path: llama-2


[2025-07-09 04:18:21] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 04:18:21] Init torch distributed begin.


[2025-07-09 04:18:21] Init torch distributed ends. mem usage=0.02 GB


[2025-07-09 04:18:22] Load weight begin. avail mem=27.17 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.44s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.44s/it]

[2025-07-09 04:18:31] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=19.13 GB, mem usage=-15.30 GB.
[2025-07-09 04:18:31] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-07-09 04:18:31] Memory pool end. avail mem=18.80 GB


[2025-07-09 04:18:31] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=17.32 GB


[2025-07-09 04:18:31] INFO:     Started server process [1413558]
[2025-07-09 04:18:31] INFO:     Waiting for application startup.
[2025-07-09 04:18:31] INFO:     Application startup complete.
[2025-07-09 04:18:31] INFO:     Uvicorn running on http://127.0.0.1:31605 (Press CTRL+C to quit)


[2025-07-09 04:18:32] INFO:     127.0.0.1:46938 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 04:18:32] INFO:     127.0.0.1:46946 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 04:18:32] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T04:18:32.826333


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-09 04:19:23] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, #token: 7, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-09T04:19:23.742048


[2025-07-09 04:19:26] INFO:     127.0.0.1:46962 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 04:19:26] The server is fired up and ready to roll!


[2025-07-09 04:19:28] INFO:     127.0.0.1:53956 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

[2025-07-09 04:19:28] Child process unexpectedly failed with exitcode=9. pid=1413786
[2025-07-09 04:19:28] Child process unexpectedly failed with exitcode=9. pid=1413720


### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 04:19:38] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=30510, nccl_port=None, mem_fraction_static=0.7, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=769679476,

[2025-07-09 04:19:45] Casting torch.bfloat16 to torch.float16.


[2025-07-09 04:19:45] Casting torch.bfloat16 to torch.float16.
[2025-07-09 04:19:46] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 04:19:46] Init torch distributed begin.


[2025-07-09 04:19:46] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 04:19:47] Load weight begin. avail mem=60.25 GB
[2025-07-09 04:19:47] The weight of LmHead is not packed


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.04it/s]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.04it/s]

[2025-07-09 04:20:03] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=58.55 GB, mem usage=1.70 GB.
[2025-07-09 04:20:03] KV Cache is allocated. #tokens: 20480, K size: 0.04 GB, V size: 0.04 GB
[2025-07-09 04:20:03] Memory pool end. avail mem=58.47 GB


[2025-07-09 04:20:04] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=59.15 GB


[2025-07-09 04:20:04] INFO:     Started server process [1417430]
[2025-07-09 04:20:04] INFO:     Waiting for application startup.
[2025-07-09 04:20:04] INFO:     Application startup complete.
[2025-07-09 04:20:04] INFO:     Uvicorn running on http://127.0.0.1:30510 (Press CTRL+C to quit)
[2025-07-09 04:20:05] INFO:     127.0.0.1:46332 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 04:20:05] INFO:     127.0.0.1:46340 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 04:20:05] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T04:20:05.862344


[2025-07-09 04:20:08] INFO:     127.0.0.1:46352 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 04:20:08] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-09 04:20:10] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T04:20:10.101321


[2025-07-09 04:20:10] INFO:     127.0.0.1:50628 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

[2025-07-09 04:20:10] Child process unexpectedly failed with exitcode=9. pid=1418024
[2025-07-09 04:20:10] Child process unexpectedly failed with exitcode=9. pid=1417817


### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 04:20:29] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='float16', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.1-8B-Instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='127.0.0.1', port=31259, nccl_port=None, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=435645043, constrai

[2025-07-09 04:20:37] Casting torch.bfloat16 to torch.float16.


[2025-07-09 04:20:37] Casting torch.bfloat16 to torch.float16.
[2025-07-09 04:20:37] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 04:20:37] Init torch distributed begin.


[2025-07-09 04:20:39] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 04:20:40] Load weight begin. avail mem=78.50 GB


...

[2025-07-09 04:20:59] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=58.56 GB


[2025-07-09 04:20:59] INFO:     Started server process [1419026]
[2025-07-09 04:20:59] INFO:     Waiting for application startup.
[2025-07-09 04:20:59] INFO:     Application startup complete.
[2025-07-09 04:20:59] INFO:     Uvicorn running on http://127.0.0.1:31259 (Press CTRL+C to quit)


[2025-07-09 04:21:00] INFO:     127.0.0.1:57306 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 04:21:00] INFO:     127.0.0.1:57316 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 04:21:00] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T04:21:00.847255


[2025-07-09 04:21:03] INFO:     127.0.0.1:57332 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 04:21:03] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-07-09 04:21:05] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T04:21:05.617254


[2025-07-09 04:21:05] INFO:     127.0.0.1:57346 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

[2025-07-09 04:21:05] Child process unexpectedly failed with exitcode=9. pid=1419243
[2025-07-09 04:21:05] Child process unexpectedly failed with exitcode=9. pid=1419175


## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 04:21:13] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='XiaomiMiMo/MiMo-7B-RL', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=39499, nccl_port=None, mem_fraction_static=0.5, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1071900191, constrained_json_whitespace_pattern=None, watc

[2025-07-09 04:21:21] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 04:21:21] Init torch distributed begin.
[2025-07-09 04:21:21] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 04:21:22] Load weight begin. avail mem=78.50 GB


[2025-07-09 04:21:22] The weight of LmHead is not packed
[2025-07-09 04:21:22] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:00<00:00,  7.60it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:00<00:00,  7.23it/s]

[2025-07-09 04:21:27] Load weight end. type=MiMoMTP, dtype=torch.bfloat16, avail mem=57.83 GB, mem usage=2.71 GB.
[2025-07-09 04:21:27] KV Cache is allocated. #tokens: 20480, K size: 1.41 GB, V size: 1.41 GB
[2025-07-09 04:21:27] Memory pool end. avail mem=54.88 GB


[2025-07-09 04:21:28] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=56.97 GB


[2025-07-09 04:21:28] INFO:     Started server process [1420055]
[2025-07-09 04:21:28] INFO:     Waiting for application startup.
[2025-07-09 04:21:28] INFO:     Application startup complete.
[2025-07-09 04:21:28] INFO:     Uvicorn running on http://0.0.0.0:39499 (Press CTRL+C to quit)
[2025-07-09 04:21:28] INFO:     127.0.0.1:38640 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 04:21:29] INFO:     127.0.0.1:38648 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 04:21:29] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T04:21:29.924815


[2025-07-09 04:21:31] INFO:     127.0.0.1:38660 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 04:21:31] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-07-09 04:21:33] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T04:21:33.996819


[2025-07-09 04:21:34] Decode batch. #running-req: 1, #token: 89, token usage: 0.00, accept len: 1.71, cuda graph: False, gen throughput (token/s): 10.93, #queue-req: 0, timestamp: 2025-07-09T04:21:34.606857


[2025-07-09 04:21:35] Decode batch. #running-req: 1, #token: 159, token usage: 0.01, accept len: 1.75, cuda graph: False, gen throughput (token/s): 106.80, #queue-req: 0, timestamp: 2025-07-09T04:21:35.262285


[2025-07-09 04:21:35] INFO:     127.0.0.1:38670 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

[2025-07-09 04:21:35] Child process unexpectedly failed with exitcode=9. pid=1420270


## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).